We initialize Python imports and opens a DuckDB connection that every later cell reuses. We also load the preferred specs and the modeling frame and set bootstrap parameters so the bootstrap can run 500 iterations without rewriting code

In [1]:
import warnings
from pathlib import Path

import duckdb
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore", category=RuntimeWarning)

CWD = Path().resolve()
DB_FILE = None
for p in [CWD] + list(CWD.parents):
    cand = p / "db" / "nflpa.duckdb"
    if cand.exists():
        DB_FILE = cand
        break
if DB_FILE is None:
    for p in [CWD] + list(CWD.parents):
        cand = p / "nflpa.duckdb"
        if cand.exists():
            DB_FILE = cand
            break
if DB_FILE is None:
    raise RuntimeError("Could not find nflpa.duckdb")

con = duckdb.connect(str(DB_FILE), read_only=False)

need = ["step18_model_frame", "step18_preferred_model_specs"]
existing = set(con.execute("SHOW TABLES").df()["name"].astype(str).tolist())
missing = [t for t in need if t not in existing]
if missing:
    raise RuntimeError(f"Missing tables for step20, {missing}, run notebook 18 first")

df = con.execute("SELECT * FROM step18_model_frame").df()
pref = con.execute("SELECT * FROM step18_preferred_model_specs").df()

TEAM_COL = "team" if "team" in df.columns else "team_key"

pref_def = pref[pref["side"] == "def"].iloc[0].to_dict()
pref_off = pref[pref["side"] == "off"].iloc[0].to_dict()

formula_def = str(pref_def["formula"])
formula_off = str(pref_off["formula"])
fam_def = str(pref_def["family"])
fam_off = str(pref_off["family"])

N_BOOT = 500
SEED = 20260106

print("bootstrap settings")
print("N_BOOT", N_BOOT)
print("SEED", SEED)
print("families", fam_def, fam_off)

bootstrap settings
N_BOOT 500
SEED 20260106
families poisson poisson


We fit the full sample preferred models once with clustered inference to produce standard confidence intervals for comparison against bootstrap intervals

In [2]:
def fit_poisson(formula: str, data: pd.DataFrame):
    m = smf.glm(formula=formula, data=data, family=sm.families.Poisson())
    r = m.fit(maxiter=200, disp=0)
    return r

def fit_nb_discrete(formula: str, data: pd.DataFrame):
    m = smf.negativebinomial(formula=formula, data=data)
    r = m.fit(disp=False, maxiter=200)
    return r

def robust_cluster(res, groups: pd.Series):
    try:
        return res.get_robustcov_results(cov_type="cluster", groups=groups)
    except Exception:
        return None

def standard_ci(res, groups: pd.Series, key_terms: list[str]) -> pd.DataFrame:
    rob = robust_cluster(res, groups)
    if rob is None:
        params = res.params
        se = res.bse
    else:
        params = rob.params
        se = rob.bse

    out = []
    for t in key_terms:
        if t in params.index:
            b = float(params.loc[t])
            s = float(se.loc[t])
            out.append({
                "term": t,
                "beta": b,
                "se_cluster": s,
                "ci_lo": b - 1.96 * s,
                "ci_hi": b + 1.96 * s,
            })
    return pd.DataFrame(out)

key_terms = [
    "shock_nonscore",
    "shock_x_blowout",
    "vol_nonscore_s2d_prior",
    "vol_nonscore_roll4_prior",
    "cum_shocks_nonscore_prior",
    "ST_Shock_NonScore_w_minus_1",
    "ST_Shock_NonScore_w_minus_2",
    "ST_Shock_NonScore_w_minus_3",
]
key_terms = [t for t in key_terms if t in df.columns or t.startswith("ST_Shock") or t.startswith("shock") or t.startswith("vol") or t.startswith("cum")]

groups = df[TEAM_COL]

res_def = None
res_off = None

try:
    res_def = fit_poisson(formula_def, df) if fam_def == "poisson" else fit_nb_discrete(formula_def, df)
except Exception as e:
    raise RuntimeError(f"Full sample defense fit failed, {str(e)}")

try:
    res_off = fit_poisson(formula_off, df) if fam_off == "poisson" else fit_nb_discrete(formula_off, df)
except Exception as e:
    raise RuntimeError(f"Full sample offense fit failed, {str(e)}")

std_def = standard_ci(res_def, groups, key_terms)
std_def["side"] = "def"
std_off = standard_ci(res_off, groups, key_terms)
std_off["side"] = "off"

std_ci_df = pd.concat([std_def, std_off], ignore_index=True)

print("standard clustered intervals for key terms")
print(std_ci_df)

standard clustered intervals for key terms
                        term      beta  se_cluster     ci_lo     ci_hi side
0             shock_nonscore  0.023231    0.030627 -0.036799  0.083261  def
1            shock_x_blowout  0.034795    0.053068 -0.069218  0.138808  def
2   vol_nonscore_roll4_prior -0.009422    0.008052 -0.025204  0.006360  def
3  cum_shocks_nonscore_prior  0.005955    0.010063 -0.013768  0.025678  def
4             shock_nonscore  0.032298    0.032283 -0.030977  0.095574  off
5            shock_x_blowout  0.073520    0.054459 -0.033221  0.180260  off
6   vol_nonscore_roll4_prior  0.023239    0.008285  0.007000  0.039477  off
7  cum_shocks_nonscore_prior -0.010309    0.010524 -0.030937  0.010318  off


Quick sanity check to confirm that the bootstrap cluster unit has the expected number of teams and that each team contributes multiple weeks so cluster resampling is meaningful

In [3]:
team_counts = df[TEAM_COL].value_counts(dropna=False).reset_index()
team_counts.columns = [TEAM_COL, "n_rows"]

print("n teams", len(team_counts))
print("min weeks per team", int(team_counts["n_rows"].min()))
print("max weeks per team", int(team_counts["n_rows"].max()))

if len(team_counts) < 20:
    raise RuntimeError("Too few teams for clustered bootstrap, something is wrong with team ids")

if int(team_counts["n_rows"].min()) < 5:
    print("warning some teams have very few rows, bootstrap stability may degrade")

n teams 35
min weeks per team 56
max weeks per team 187


We run the clustered bootstrap by sampling teams with replacement and retaining all weeks for each sampled team and then refit Models A and B each iteration with guarded error handling

In [4]:
rng = np.random.default_rng(SEED)
teams = df[TEAM_COL].dropna().astype(str).unique().tolist()
n_teams = len(teams)

def build_boot_df(sampled_teams: np.ndarray) -> pd.DataFrame:
    vc = pd.Series(sampled_teams).value_counts()
    parts = []
    for t, k in vc.items():
        sub = df[df[TEAM_COL].astype(str) == str(t)]
        if len(sub) == 0:
            continue
        if int(k) == 1:
            parts.append(sub)
        else:
            parts.append(pd.concat([sub] * int(k), ignore_index=True))
    if len(parts) == 0:
        return df.iloc[0:0].copy()
    return pd.concat(parts, ignore_index=True)

def extract_key_params(res, side: str, iter_id: int, key_terms: list[str]) -> list[dict]:
    params = res.params.copy()
    out = []
    for t in key_terms:
        if t in params.index:
            out.append({
                "iter": iter_id,
                "side": side,
                "term": t,
                "beta": float(params.loc[t]),
            })
        else:
            out.append({
                "iter": iter_id,
                "side": side,
                "term": t,
                "beta": np.nan,
            })
    return out

boot_rows = []
fail_rows = []

for b in range(1, N_BOOT + 1):
    sampled = rng.choice(teams, size=n_teams, replace=True)
    bdf = build_boot_df(sampled)

    ok_def = 1
    ok_off = 1

    r_def = None
    r_off = None

    try:
        r_def = fit_poisson(formula_def, bdf) if fam_def == "poisson" else fit_nb_discrete(formula_def, bdf)
    except Exception as e:
        ok_def = 0
        fail_rows.append({"iter": b, "side": "def", "error": str(e)})

    try:
        r_off = fit_poisson(formula_off, bdf) if fam_off == "poisson" else fit_nb_discrete(formula_off, bdf)
    except Exception as e:
        ok_off = 0
        fail_rows.append({"iter": b, "side": "off", "error": str(e)})

    if r_def is not None:
        boot_rows.extend(extract_key_params(r_def, "def", b, key_terms))
    if r_off is not None:
        boot_rows.extend(extract_key_params(r_off, "off", b, key_terms))

    if b % 25 == 0:
        print("bootstrap iter", b, "fail def", int(sum(1 for x in fail_rows if x["side"] == "def")), "fail off", int(sum(1 for x in fail_rows if x["side"] == "off")))

boot_df = pd.DataFrame(boot_rows)
fail_df = pd.DataFrame(fail_rows)

print("bootstrap coefficient rows", len(boot_df))
print("bootstrap failures", len(fail_df))
if len(fail_df) > 0:
    print(fail_df.head(10))

bootstrap iter 25 fail def 0 fail off 0
bootstrap iter 50 fail def 0 fail off 0
bootstrap iter 75 fail def 0 fail off 0
bootstrap iter 100 fail def 0 fail off 0
bootstrap iter 125 fail def 0 fail off 0
bootstrap iter 150 fail def 0 fail off 0
bootstrap iter 175 fail def 0 fail off 0
bootstrap iter 200 fail def 0 fail off 0
bootstrap iter 225 fail def 0 fail off 0
bootstrap iter 250 fail def 0 fail off 0
bootstrap iter 275 fail def 0 fail off 0
bootstrap iter 300 fail def 0 fail off 0
bootstrap iter 325 fail def 0 fail off 0
bootstrap iter 350 fail def 0 fail off 0
bootstrap iter 375 fail def 0 fail off 0
bootstrap iter 400 fail def 0 fail off 0
bootstrap iter 425 fail def 0 fail off 0
bootstrap iter 450 fail def 0 fail off 0
bootstrap iter 475 fail def 0 fail off 0
bootstrap iter 500 fail def 0 fail off 0
bootstrap coefficient rows 8000
bootstrap failures 0


We compute bootstrap means and percentile confidence intervals for each key coefficient and compare them directly against the standard clustered intervals

In [5]:
def summarize_boot(side: str) -> pd.DataFrame:
    sub = boot_df[boot_df["side"] == side].copy()
    out = []
    for t in sorted(sub["term"].unique().tolist()):
        vals = sub[sub["term"] == t]["beta"].astype(float).to_numpy()
        vals = vals[np.isfinite(vals)]
        if len(vals) == 0:
            out.append({"side": side, "term": t, "n_eff": 0, "boot_mean": np.nan, "boot_ci_lo": np.nan, "boot_ci_hi": np.nan})
            continue
        out.append({
            "side": side,
            "term": t,
            "n_eff": int(len(vals)),
            "boot_mean": float(np.mean(vals)),
            "boot_ci_lo": float(np.quantile(vals, 0.025)),
            "boot_ci_hi": float(np.quantile(vals, 0.975)),
        })
    return pd.DataFrame(out)

boot_sum_def = summarize_boot("def")
boot_sum_off = summarize_boot("off")
boot_sum = pd.concat([boot_sum_def, boot_sum_off], ignore_index=True)

comp = boot_sum.merge(
    std_ci_df.rename(columns={"beta": "std_beta", "ci_lo": "std_ci_lo", "ci_hi": "std_ci_hi"}),
    on=["side", "term"],
    how="left",
)

print("bootstrap versus standard interval comparison")
print(comp.sort_values(["side", "term"]).reset_index(drop=True))

if len(fail_df) > 0:
    fail_rate_def = float((fail_df["side"] == "def").mean())
    fail_rate_off = float((fail_df["side"] == "off").mean())
    print("failure rate def", fail_rate_def)
    print("failure rate off", fail_rate_off)

bootstrap versus standard interval comparison
   side                         term  n_eff  boot_mean  boot_ci_lo  \
0   def  ST_Shock_NonScore_w_minus_1      0        NaN         NaN   
1   def  ST_Shock_NonScore_w_minus_2      0        NaN         NaN   
2   def  ST_Shock_NonScore_w_minus_3      0        NaN         NaN   
3   def    cum_shocks_nonscore_prior    500   0.005822   -0.022689   
4   def               shock_nonscore    500   0.025099   -0.032977   
5   def              shock_x_blowout    500   0.032426   -0.065369   
6   def     vol_nonscore_roll4_prior    500  -0.009791   -0.024600   
7   def       vol_nonscore_s2d_prior      0        NaN         NaN   
8   off  ST_Shock_NonScore_w_minus_1      0        NaN         NaN   
9   off  ST_Shock_NonScore_w_minus_2      0        NaN         NaN   
10  off  ST_Shock_NonScore_w_minus_3      0        NaN         NaN   
11  off    cum_shocks_nonscore_prior    500  -0.010172   -0.037754   
12  off               shock_nonscore    500 